[![Open In Wolfram Cloud](https://raw.githubusercontent.com/gvarnavi/generative-art-iap/master/PR/wolfram-badge.svg)](https://www.wolframcloud.com/obj/gvarnavi/Published/03X_wallpaper-groups.nb)

# Wallpaper Groups

So far, we've examined a strange attractors exhibiting specific point group symmetries.  
We can go further by enforcing our functions also preserve translational symmetry and allowing for glide planes.

We will differentiate between two fundamental lattice types in 2D: square and hexagonal  
**Note**: Keen readers will be alarmed we've missed three of the five Bravais lattices in 2D - rest assured, we'll recover those using our definition of a translation group!

In [ ]:
latticeVectors["square"] = {{1, 0}, {0, 1}};
latticeVectors["hexagonal"] = {{1, 0}, {-1/2, Sqrt[3]/2}};

Let's quickly add points at each lattice site to see how these look like.  
First, we can make a square grid at integer combinations of lattice vectors

In [ ]:
Tuples[Range[-3, 3], 2]

Then, we "stretch" these using our lattice vectors by dotting them on the right

In [ ]:
Tuples[Range[-3, 3], 2] . latticeVectors["hexagonal"]

Finally, we add a Point for each lattice site and wrap the expression in Graphics

In [ ]:
Tuples[Range[-3, 3], 2] . latticeVectors["hexagonal"] // Point // Graphics

We'll also need the dual or inverse lattice to this.  
This is given by:
$$
\left[ b_1 , b_2 , b_3 \right]^T = 2\pi \left[a_1, a_2, a_3 \right]^{-1}
$$

In [ ]:
reciprocalVectors["square"] = 2 \[Pi] Transpose[Inverse[latticeVectors["square"]]];
reciprocalVectors["hexagonal"] = 2 \[Pi] Transpose[Inverse[latticeVectors["hexagonal"]]];

## Symmetries
### Translation Group

We can ensure a function is translationally invariant, by using the periodic sinusoidal functions `Sin` and `Cos` as $f_1,f_2,f_3,f_4$ according to:

$$ \begin{pmatrix}x \\ y \end{pmatrix} =
\begin{pmatrix}
a f_1\left[\sum_{v \in V} f_2[(x,y).v] + (x,y).v \right] \\
b f_3\left[\sum_{v \in V} f_4[(x,y).v] + (x,y).v \right]
\end{pmatrix}
$$

where $V$ is any finite subset of lattice vectors 

In [ ]:
translationGroup[latticeVectors_][a_, b_, {f1_, f2_, f3_, f4_}][{x_, y_}] :=
 {
  a f1[Sum[f2[{x, y} . v] + {x, y} . v, {v, latticeVectors}]],
  b f3[Sum[f4[{x, y} . v] + {x, y} . v, {v, latticeVectors}]]
 }

Let's test this!

In [ ]:
FullSimplify[
 translationGroup[latticeVectors["hexagonal"]][a, b, {Sin, Cos, Cos, Sin}]
 [{x, y}]]
 
FullSimplify[
 translationGroup[latticeVectors["hexagonal"]][a,b, {Sin, Cos, Cos, Sin}]
 [{x, y} + RandomInteger[{-4, 4}, 2].reciprocalVectors["hexagonal"]]]

### Point Group
In 2D, we only have cylic ($C_n$) and dihedral ($D_n$) symmetries:

In [ ]:
cyclic[n_] := 
 With[{base = 
 {{Cos[2 Pi/n], -Sin[2 Pi/n]}, {Sin[2 Pi/n],Cos[2 Pi/n]}}},
  AffineTransform[MatrixPower[base, #]] & /@ Range[n]]
  
cyclic[4]

In [ ]:
dihedral[n_] := 
 With[
 {base = {{Cos[2 Pi/n], -Sin[2 Pi/n]}, {Sin[2 Pi/n],Cos[2 Pi/n]}}},
  Union[cyclic[n], 
   AffineTransform[{{-1, 0}, {0, 1}} . MatrixPower[base, #]] & /@ 
    Range[n]]]
    
dihedral[4]

### 'Extra' Symmetries
Plane groups can also exhibit `glide` symmetry, a combination of translation and reflection.

We define these explicitly for the 17 wallpaper groups:

In [ ]:
planeGroupSymmetries =
  <|"p1" -> <|"Lattice" -> latticeVectors["square"],
     "Group Symmetries" -> cyclic[1],
     "Extra Symmetries" -> {}|>,
   
   "p2" -> <|"Lattice" -> latticeVectors["square"],
     "Group Symmetries" -> cyclic[2], "Extra Symmetries" -> {}|>,
   
   "p1m1" -> <|"Lattice" -> latticeVectors["square"],
     "Group Symmetries" -> dihedral[1], 
     "Extra Symmetries" -> {ReflectionTransform[{0, 1}]}|>,
   
   "p1g1" -> <|"Lattice" -> latticeVectors["square"],
     "Group Symmetries" -> dihedral[1], 
     "Extra Symmetries" -> {TranslationTransform[{Pi, 0}]@*
        ReflectionTransform[{0, 1}]}|>,
   
   "c1m1" -> <|"Lattice" -> latticeVectors["square"],
     "Group Symmetries" -> dihedral[1], 
     "Extra Symmetries" -> {ReflectionTransform[{0, 1}], 
       TranslationTransform[{Pi, Pi}]@*
        ReflectionTransform[{0, 1}]}|>,
   
   "p2mm" -> <|"Lattice" -> latticeVectors["square"],
     "Group Symmetries" -> dihedral[2], 
     "Extra Symmetries" -> {ReflectionTransform[{0, 1}], 
       ReflectionTransform[{1, 0}]}|>,
   
   "p2gg" -> <|"Lattice" -> latticeVectors["square"],
     "Group Symmetries" -> dihedral[2], 
     "Extra Symmetries" -> {TranslationTransform[{Pi, Pi}]@*
        ReflectionTransform[{0, 1}], 
       TranslationTransform[{Pi,Pi}]@*
        ReflectionTransform[{1, 0}]}|>,
   
   "p2mg" -> <|"Lattice" -> latticeVectors["square"],
     "Group Symmetries" -> dihedral[2], 
     "Extra Symmetries" -> {TranslationTransform[{Pi, 0}]@*
        ReflectionTransform[{0, 1}], ReflectionTransform[{1, 0}]}|>,
   
   "c2mm" -> <|
     <|"Lattice" -> latticeVectors["square"],
      "Group Symmetries" -> dihedral[2], 
      "Extra Symmetries" -> {ReflectionTransform[{1, 0}], 
        ReflectionTransform[{0, 1}], 
        TranslationTransform[{Pi, Pi}]@*
         ReflectionTransform[{0, 1}], 
        TranslationTransform[{Pi, Pi}]@*
         ReflectionTransform[{1, 0}]}|>|>,
   
   "p4" -> <|"Lattice" -> latticeVectors["square"],
     "Group Symmetries" -> cyclic[4], "Extra Symmetries" -> {}|>,
   
   "p4mm" -> <|"Lattice" -> latticeVectors["square"],
     "Group Symmetries" -> dihedral[4], 
     "Extra Symmetries" -> {ReflectionTransform[{0, 1}]}|>,
   
   "p4gm" -> <|"Lattice" -> latticeVectors["square"],
     "Group Symmetries" -> dihedral[4], 
     "Extra Symmetries" -> {TranslationTransform[{Pi, 0}]@*
        ReflectionTransform[{0, 1}]}|>,
   
   "p3" -> <|"Lattice" -> latticeVectors["hexagonal"],
     "Group Symmetries" -> cyclic[3], "Extra Symmetries" -> {}|>,
   
   "p3m1" -> <|"Lattice" -> latticeVectors["hexagonal"],
     "Group Symmetries" -> dihedral[3], 
     "Extra Symmetries" -> {ReflectionTransform[{1, 0}]}|>,
   
   "p31m" -> <|"Lattice" -> latticeVectors["hexagonal"],
     "Group Symmetries" -> dihedral[3], 
     "Extra Symmetries" -> {ReflectionTransform[{0, 1}]}|>,
   
   "p6" -> <|"Lattice" -> latticeVectors["hexagonal"],
     "Group Symmetries" -> cyclic[6], "Extra Symmetries" -> {}|>,
   
   "p6m" -> <|"Lattice" -> latticeVectors["hexagonal"],
     "Group Symmetries" -> dihedral[6], 
     "Extra Symmetries" -> {ReflectionTransform[{0, 1}]}|>
   |>;

For example, for wallpaper group p4gm, we consider the additional glide plane:

In [ ]:
planeGroupSymmetries["p4gm"]

### Function Equivariance
We're looking to generate strange attractor functions which, when nested, produce the desired plane-group symmetry. We therefore need to define what it means for a function to have a particular symmetry.

We'll define a function `f` to be **equivariant** with respect to symmetry $\sigma$ if $f(\sigma(x)) = \sigma(f(x))$, i.e. the function and the symmetry operation commute.

Putting this all-together, we arrive at our `wallpaperGroup` function, which takes the amplitudes, `a`, `b`, periodic functions $f_1,f_2,f_3,f_4$, and the wallpaper group - and returns a functional form exhibiting that symmetry: 

In [ ]:
Clear[wallpaperGroup]
wallpaperGroup[wallpaper_][a_, b_, {f1_, f2_, f3_, f4_}][r_] := 
 wallpaperGroup[wallpaper][a, b, {f1, f2, f3, f4}][r] = With[{
    latticeVectors  = planeGroupSymmetries[wallpaper, "Lattice"],
    groupSymmetries = planeGroupSymmetries[wallpaper, "Group Symmetries"], 
    extraSymmetries = planeGroupSymmetries[wallpaper, "Extra Symmetries"]
    },
   Simplify[
    Sum[translationGroup[latticeVectors][a, b, {f1, f2, f3, f4}][
        g[r]], {g, groupSymmetries}]/Length[groupSymmetries]
    +
     Sum[translationGroup[latticeVectors][a, 
         b, {f1, f2, f3, f4}][(\[Sigma]@*g)[
         r]]/(Length[extraSymmetries] Length[groupSymmetries]), {g, 
       groupSymmetries}, {\[Sigma], extraSymmetries}]]
]

wallpaperGroup["p4gm"][a, b, {Sin,Cos,Cos,Sin}][{x,y}]

## Visualization Scheme
Now that we have our general symmetry-preserving chaotic attractors, we can follow a similar scheme as in the strange attractor notebook to visualize these.

To extenuate the fractal nature of these maps, we take this opportunity to demonstrate another visualization scheme called `Orbit Traps`.

According to [Wikipedia](https://en.wikipedia.org/wiki/Orbit_trap):
>An orbit trap is a method of colouring fractal images based upon how close an iterative function, used to create the fractal, approaches a geometric shape, called a "trap".

We will use point traps. In order to preserve a wallpaper groups' periodic nature, we will use periodic point traps as-well.

In [ ]:
orbitTraps[radius_] := 
<|
  1 -> {{0., 0.}}, 
  2 -> CirclePoints[2/Sqrt[3] radius, 3], 
  3 -> CirclePoints[Sqrt[2] radius, 4], 
  4 -> Prepend[CirclePoints[{2 radius, Pi/6}, 6], {0., 0.}]
|>

Multicolumn[
 Table[Graphics[Circle /@ orbitTraps[1][i], Axes -> True, 
   Ticks -> None,ImageSize->100], {i, 4}], 2, Frame -> All]

Essentially, we'll keep iterating our wallpaper strange attractor - until our function 'falls' in one of our traps, and then return the normalized distance of that point to the center of the trap 

## Compiled Strange Attractors
Like before, we'll compile our iteration code in `C` to get a reasonable speed-up!

We define our main logic as a function, which takes our lattice function, and our particular orbit trap:

In [ ]:
compilerFunction[latticeFunc_, costFunc_] := Compile[{
   
   {xmin, _Real}, {xmax, _Real},
   {ymin, _Real}, {ymax, _Real},
   {delta, _Real},
   {itmax, _Integer}, {escapeRadius, _Real},
   {a, _Real}, {b, _Real}},
  
  Block[{iters, x, y, cost},
   Table[
    iters = 0; {x, y} = {rx, ry}; cost = 1.0;
    
    While[(iters < itmax) && (cost >= 1),
     iters++;
     {x, y} = latticeFunc;
     cost = costFunc;
     ];
    
    cost
    ,
    {rx, xmin, xmax, delta},
    {ry, ymin, ymax, delta}]],
  
  CompilationTarget -> "C", RuntimeOptions -> "Speed", 
  CompilationOptions -> {"ExpressionOptimization" -> False}]

Pre-compute all $f_1,f_2,f_3,f_4$ function permutations for each wallpaper group:

In [ ]:
sinusoids = Tuples[{Sin, Cos}, 4];
precomputedLatticeFunctions = 
   Association[
    Table[i -> 
      DeleteCases[
       Association[
        Table[j -> 
          wallpaperGroup[i][a, b, sinusoids[[j]]][{x, y}], {j, 16}]], 
          {0, 0} | {0, _} | {_, 0}], 
          {i,Keys[planeGroupSymmetries]}]];

Pre-compute all orbit trap functions:

In [ ]:
precomputedCostFunctions = 
 Association[
  Table[i -> 
    Min[Table[
       Norm[pt escapeRadius - {x, y}], {pt, orbitTraps[1][i]}]]/
     escapeRadius, {i, 4}]];

And finally, write a wrapper function to compile the iterator for a particular wallpaper group using a `RandomChoice` of $f_1,f_2,f_3,f_4$

In [ ]:
compilerWrapper[planeGroup_, orbitTrap_] := 
 compilerFunction[
  RandomChoice[precomputedLatticeFunctions[planeGroup]], 
  precomputedCostFunctions[orbitTrap]
  ]

cf["p2mg"] = compilerWrapper["p2mg", 1]

## Visualization Function
Finally, we write a visualization function, using some built-in color schemes:

In [ ]:
Clear[visualizeWallpaper]
visualizeWallpaper[wallpaperGroup_, orbitTrap_ , colFunction_] := 
With[
{
iterator=compilerWrapper[wallpaperGroup, orbitTrap],
range = N[4 Pi]
},
  ArrayPlot[
   Log[iterator[-range, range, -range, range, .1, 100, 2.5, 
      RandomReal[{0., 3.}], RandomReal[{0., 3.}]] + 1], 
   ColorFunction -> colFunction, ImagePadding -> None, 
   PlotRangePadding -> None, ImageSize -> 200,Frame -> False, 
   PlotLabel -> Style[wallpaperGroup, 16, Black]]
]

In [ ]:
visualizeWallpaper["p3m1", 2, "GrayYellowTones"]

In [ ]:
visualizeWallpaper["c2mm", 4, ColorData[{"AvocadoColors","Reversed"}]]

## Your turn!
Play around with some of the wallpaper groups and orbit traps.  
As a reminder, here are the 17 wallpaper group names, and some built-in colorfunctions:

In [ ]:
Keys[planeGroupSymmetries]

In [ ]:
ColorData["Gradients"]